# Natural Language Processing

Start met laden van de packages en onze zojuist gemaakte dataset

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/fiets.csv', index_col =0)

In [ ]:
data.head(2)

In [ ]:
len(data)

## Count word frequencies

Als eerste, meeste simpele stap van tekst analyse, kunnen we kijken naar de meest voorkomende woorden in de artikelen. Onderstaande functie is geschreven om dit te doen. Maar als eerste stap moeten we de artikelen van de kolom 'content' omzetten naar een lijst met woorden per artikel. 

In [ ]:
data['content list'] = data.content.apply(lambda x: x.lower().split())

In [ ]:
def word_counter(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

We kunnen de functie uitproberen:

In [ ]:
word_counter(data['content list'])

Zoals je ziet zijn de eerste woorden allemaal woorden die inhoudelijk niet veel zeggen over de artikelen. Deze woorden, die veel voorkomen in onze taal maar inhoudelijk weinig zinnigs zeggen, noemen we stopwoorden. We kunnen een stopwoordenlijst gebruiken om deze woorden weg te filteren. 

In [ ]:
with open('data/stopwords-nl.txt', 'r') as f:
    stopwords = f.read().split("\n")

In [ ]:
## Don't alter
def word_counter_stopword(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
        
    full_list = [i for i in full_list if i not in stopwords]
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

In [ ]:
word_counter_stopword(data['content list'])

In [ ]:
word_counter_stopword(data['content list']).head(10)

# Spacy

Naast een naieve worcount, zijn er ook andere opties mogelijk. Voor het vervolg van deze workshop gaan we gebruik maken van de Spacy package, die gebruikt wordt voor Natural language processing. 


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import pickl

In [ ]:
##nlp = spacy.load("nl_core_news_sm")

In [ ]:
##data = data.dropna(subset=['content'])

In [ ]:
#def process_text(text):
 #   return nlp(text)

#def flatten(xss):
 #   return [x for xs in xss for x in xs]

In [ ]:
## deze code als comment omdat het lang duurt om te draaien
#data["doc"] = data["content"].apply(process_text)

In [ ]:
## Deze code ook als comment om te laten zien hoe je het model kan opslaan
##import pickle

#with open('data/fiets_nlp.pkl', 'wb') as f:
 #   pickle.dump(data, f)

In [ ]:
with open('data/fiets_nlp.pkl', 'rb') as f:
    fiets_nlp = pickle.load(f)

In [ ]:
fiets_nlp.head(2)

# Named Entity Recognition

Spacy slaat informatie over de gevonden named entities op in het doc item. Deze informatie staat opgeslagen in de kolom 'doc' en kan daar ook uitgehaald worden.

Spacy kent de volgende Named entities:

* PERSON:      People, including fictional.
* NORP:        Nationalities or religious or political groups.
* FAC:         Buildings, airports, highways, bridges, etc.
* ORG:         Companies, agencies, institutions, etc.
* GPE:         Countries, cities, states.
* LOC:         Non-GPE locations, mountain ranges, bodies of water.
* PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
* EVENT:       Named hurricanes, battles, wars, sports events, etc.
* WORK_OF_ART: Titles of books, songs, etc.
* LAW:         Named documents made into laws.
* LANGUAGE:    Any named language.
* DATE:        Absolute or relative dates or periods.
* TIME:        Times smaller than a day.
* PERCENT:     Percentage, including ”%“.
* MONEY:       Monetary values, including unit.
* QUANTITY:    Measurements, as of weight or distance.
* ORDINAL:     “first”, “second”, etc.
* CARDINAL:    Numerals that do not fall under another type.

Je kan ook de spacy module vragen voor uitleg over een bepaalde Named Entity:

In [ ]:
spacy.explain('GPE')

Je kan per artikel de gevonden Named Entities tonen. Deze worden dan met verschillende kleuren weergegeven. 

In [ ]:
doc = fiets_nlp['doc'].to_list()[2]

In [ ]:
displacy.render(doc, style = "ent")

In [ ]:
## Zelf doen: Bekijk een random andere regel

Naast de named entities op deze manier tonen, kun je er ook verdere analyses mee doen. Je kan ze bijvoorbeeld per category opslaan als losse kolommen in je dataframe

Hieronder staat een functie waarmee je Named entities van een bepaalde soort uit het doc item kan halen. Deze worden vervolgens als een lijst opgeslagen in een nieuwe kolom in het dataframe. 

In [ ]:
def get_ner(doc, entity):
    return [ent.text for ent in doc.ents if ent.label_ == entity]

Vervolgens kun je deze functie gebruiken op je dataframe. 
Deze functie heeft als input de doc item van het dataframe, en de gekozen named entity.

In onderstaande voorbeeld wordt de categroy GPE eruit gehaald. 

In [ ]:
fiets_nlp['GPE'] = fiets_nlp['doc'].apply(lambda x: get_ner(x, 'GPE'))

Vervolgens kun je deze kolom weer gebruiken om de meest voorkomende GPE eruit te halen. 

In [ ]:
word_counter(fiets_nlp['GPE']).head(10)

In [ ]:
## Zelf doen: nieuwe kolom named entity
## Gebruik de volgende code: fiets_nlp['naam nieuwe kolom'] = fiets_nlp['doc'].apply(lambda x: get_ner(x, 'naam entity'))
## 

In [ ]:
## Word counter: word_counter(fiets_nlp['nieuwe kolom']).head(10)

In [ ]:
## Extra opgave: kies nog een aantal entities uit en bekijk de top 10. Kan je ook de top 5 of top 25 maken?